# Using a CNN for news topic categorization

In [1]:
# set this to true to learn the model when running. false will load the model from disk
LEARN=False

In [63]:
from gensim.models import Word2Vec
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Merge
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from pandas import DataFrame
import numpy as np
from sklearn.metrics import confusion_matrix

np.random.seed(0)

In [14]:
category_names = ['Sonstiges', 'Lifestyle', 
          'Wirtschaft', 'Finanzen', 'Ausland', 'Lokal', 
          'Politik', 'Sport', 'Technologie', 'Kultur']

num_models = len(category_names)

# the list of full corpora
fulldata_paths = [(x, "corpus/corpus{}.txt".format(x)) for x in category_names]

# the corpora with a fixed split for training and validation
train_paths = [(x, "data/corpus{}.training.txt".format(x)) for x in category_names]
validation_paths = [(x, "data/corpus{}.validation.txt".format(x)) for x in category_names]

base_model = Word2Vec.load('../wiki/data/wiki.de.200dim.word2vec.model')

# we can precompute the L2-normalized vectors to save lots of memory
# we can't continue learning after they are normalized but the model is static
# in this usecase anyways
base_model.init_sims(replace=True)

k = base_model.vector_size
print("basemodel has {} dimensional vectors".format(k))

basemodel has 200 dimensional vectors


**TODO:** stopword filtering is commented out, check if filtering stopwords improves perfomance, but since the cnn learns "patterns" the filtering may distort the pattern too much

In [89]:
def load_sets(paths):
    X, y = [], []
    for name, path in paths:
        articles_loaded = 0
        with open(path) as cur_file:
            for line in cur_file:
                tokens = [x for x in line.split()]# if x not in stopwords]
                if len(tokens) > 0:
                    X.append(tokens)
                    y.append(name)

    print("loaded {} articles".format(len(X)))
    return X, y

In [101]:
def equalize_a_posteriori(X, y):
    # get the a posteriori for each class:
    a_posteriori = {x: y.count(x) for x in set(y)}
    print("p(C) for all classes (not normalized):\n")
    for k,v in a_posteriori.iteritems():
        print("{:<15}:{:>5}".format(k, v))
    max_posteriori = max(a_posteriori.values())
    min_posteriori = min(a_posteriori.values())
    
    print("equalizing all training classes to argmin(p(C)) = {}".format(min_posteriori))
    
    equal_X = []
    equal_y = []
    current_aposterioris = {c: 0 for c in a_posteriori.keys()}
    
    #sort the list descending by length of the article. this way we get rid of most of the padding
    sorted_Xy = sorted(zip(X, y), key=lambda x: len(x[0]), reverse=True)
    for cur_X, cur_y in sorted_Xy:
        if(current_aposterioris[cur_y] <= min_posteriori):
            equal_X.append(cur_X)
            equal_y.append(cur_y)
            current_aposterioris[cur_y] += 1 
            
    return equal_X, equal_y

load the raw train and validation datasets in string format

In [102]:
raw_train_X, raw_train_y = load_sets(train_paths)
raw_validation_X, raw_validation_y = load_sets(validation_paths)

raw_train_X, raw_train_y = equalize_a_posteriori(raw_train_X, raw_train_y)
print("after equalization, there are {} articles for training left".format(len(raw_train_X)))

loaded 13140 articles
loaded 5594 articles
p(C) for all classes (not normalized):

Sonstiges      : 1789
Lifestyle      :  775
Wirtschaft     : 1960
Finanzen       :  763
Ausland        : 1122
Lokal          :  583
Politik        : 3772
Sport          : 1047
Technologie    :  837
Kultur         :  492
equalizing all training classes to argmin(p(C)) = 492
after equalization, there are 4929 articles for training left


calculate the average article length $N$

$$N_{avg}=\frac { \sum _{ x\in X }^{  }{ dim(x) }  }{ dim(X) } $$

In [103]:
N_avg = sum([len(article) for article in raw_train_X]) / len(raw_train_X)
print("average article length is: {} words".format(N_avg))

#override to avoid out-of-mem errors while learning
N_avg = 200

padding = [len(article) - N_avg for article in raw_train_X if len(article) - N_avg < 0]
average_padding = -sum(padding) / len(padding)
print("{} of {} articles get padded ({}%)".format(len(padding), len(raw_train_X), len(padding) * 100.0 / len(raw_train_X)))
print("average padding is {} words".format(average_padding))

average article length is: 827 words
44 of 4929 articles get padded (0.892675999188%)
average padding is 41 words


convert the raw input article to matrices of word-vectors.
each article $x$ is represented as

$${ x }_{ 1:n }={ x }_{ 1 }\oplus { x }_{ 2 }\oplus \dots \oplus { x }_{ n }$$

where:
* ${ x }_{ i }\in { { R } }^{ k }$ is the $k$-dimensional word-embedding vector for the $i$-th word in the article.
* $\oplus$ is the concatenation operator

the result is a matrix for each article in the form:
$$x=\begin{bmatrix} { x }_{ 1,1 } & { x }_{ 1,2 } & \cdots  & { x }_{ 1,k } \\ { x }_{ 2,1 } & { x }_{ 2,2 } & \cdots  & { x }_{ 2,k } \\ \vdots  & \vdots  & \ddots  & \vdots  \\ { x }_{ n,1 } & { x }_{ n,2 } & \cdots  & { x }_{ n,k } \end{bmatrix}$$

where:
* ${ x }_{ n,k }$ is the value of the $k$-th dimension of the word-vector for word $n$

the matrix is padded or cropped to a length of $N_{avg}$

In [113]:
def articles_to_matrices(articles, word_dim, article_len):
    X = np.zeros((len(articles), article_len, word_dim), dtype='float32')
    words_found = 0
    words = 0
    for x, raw_article in enumerate(articles):    
        for x_n in range(N_avg):#while (words_found < N_avg):
            # if the word can't be found, use a randomly initialized vector
            # with the same distribution as the other vectors in the model
            stddev = np.std(X)
            mean = np.mean(X)
            word_vec = np.random.normal(mean, stddev, word_dim)

            # try to load the word from the basemodel
            # TODO: maybe skip non-available words rather than default-zero
            # so if more then N_avg words ar in the article they get used
            try:
                word_vec = base_model[raw_article[x_n % len(raw_article)]]
                words_found += 1
                X[x, x_n] = word_vec
            except:
                pass
            words += 1
    
    words_skipped = words - words_found
    print("skipped {} of {} words ({}%)".format(words_skipped, words, words_skipped*100.0 / words))
    return X

train_X = articles_to_matrices(raw_train_X, k, N_avg)
validation_X = articles_to_matrices(raw_validation_X, k, N_avg)

ValueError: scale <= 0

convert the string train input data to a one-hot vector that can be used on the output layer of the cnn

In [105]:
def categories_to_one_hot(categories):
    category_names, int_y = np.unique(categories, return_inverse=True)
    y = np_utils.to_categorical(int_y)
    return y, category_names

train_y, _ = categories_to_one_hot(raw_train_y)
target_y, category_names = categories_to_one_hot(raw_validation_y)

stat = np.zeros(num_models, dtype=int)
for row in target_y:
    stat[np.argmax(row)] += 1

validation_stats = DataFrame(stat, category_names, ['dim(V_i)'])
print(validation_stats)

             dim(V_i)
Ausland           483
Finanzen          333
Kultur            208
Lifestyle         328
Lokal             249
Politik          1596
Sonstiges         770
Sport             449
Technologie       361
Wirtschaft        817


Build the model from (Convolutional Neural Networks for Sentence Classification)[https://arxiv.org/abs/1408.5882]

**Notes:**
* Don't use L2 norm contraints on weight vectors (see (A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification)[https://arxiv.org/abs/1510.03820]) (info from http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow)
* the input has 3 filter branches
* the fully connected layer has ~~one~~ no hidden layer

**Architecture** (from original paper):
![Architecture](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM.png)

In [106]:
# number of filters of each size
num_filters = 128
# square filter sizes (3x3, 4x4, and 5x5)
filter_sizes = [3, 4, 5]
num_filter_branches = len(filter_sizes)

# add the channel dimension (only 1 channel)
# tip with np.expand_dims is from http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow
train_X = np.expand_dims(train_X, -1)
validation_X = np.expand_dims(validation_X, -1)

# create the filter branches
filter_branches = []
for i, filter_size in enumerate(filter_sizes):
    branch = Sequential()
    branch.add(Convolution2D(num_filters, filter_size, k, init='uniform', border_mode='same',
                        input_shape=train_X.shape[1:], W_regularizer=l2(0.01)))
    branch.add(Activation('relu'))
    pool_size =  num_filters;
    branch.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
    #branch.add(Dropout(0.25))
    filter_branches.append(branch)

# merge the branches by concatenating 
merged_filters = Merge(filter_branches, mode='concat')

#create the final model with the filter layers and the fully connected layers
model = Sequential()
model.add(merged_filters)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_models))
model.add(Activation('softmax'))

# compile the model with an accuracy measurement
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['acc'])

In [107]:
callbacks = []
callbacks.append(TensorBoard(log_dir='logs', histogram_freq=0, write_graph=True, write_images=False))
callbacks.append(ModelCheckpoint('checkpoints', monitor='acc', verbose=1, save_best_only=True, mode='max'))
callbacks.append(EarlyStopping(monitor='loss', min_delta=0.01, patience=3, verbose=1, mode='auto'))

train the model

In [108]:
# since we have more than one filter branch, tile the input data to all branches
# np.tile causes the numpy kernel to crash (probably due to copying of the data. 
# putting it in a list works, because it then is just referenced 3 times)

#tiled_train_X = np.tile(expanded_train_X, (num_filter_branches, 1))
#tiled_validation_X = np.tile(expanded_validation_X, (num_filter_branches, 1))

tiled_train_X = [train_X, train_X, train_X]
tiled_validation_X = [validation_X, validation_X, validation_X]

if LEARN:
    model.fit(tiled_train_X, train_y, 
              validation_data=(tiled_validation_X, target_y),
              nb_epoch=1, batch_size=128,
              callbacks=callbacks)

Train on 4929 samples, validate on 5594 samples
Epoch 1/1
4929/4929 [==============================] - 29098s - loss: 2.2896 - acc: 0.1384 - val_loss: 2.2588 - val_acc: 0.1566


In [109]:
if LEARN:
    model.save('data/news.cnn.same-a-priori.model')
else:
    from keras.models import load_model
    model = load_model('data/news.cnn.model')

In [110]:
predicted_y = model.predict([validation_X, validation_X, validation_X])

In [111]:
# TODO: weight the output by the number of elements.
# Why? because it may work although it won't make any sense...
#predicted_y *= max(stat)
#predicted_y /= stat

#convert the log likelyhood prediction to a single-hot vector
# with hotspot at index of highest likelihood
predicted_y_singlehot = np.zeros(predicted_y.shape)
for x, row in enumerate(predicted_y):
    max_index = np.where(row == max(row))
    predicted_y_singlehot[x, max_index] = 1
    

In [112]:
classification_matrix = np.zeros([num_models, num_models], dtype=int)

for i in range(len(predicted_y_singlehot)):
    predicted_index = np.where(predicted_y_singlehot[i] == 1)[0]
    target_index = np.where(target_y[i] == 1)[0]
    classification_matrix[predicted_index, target_index] += 1

result = DataFrame(classification_matrix, category_names, category_names)
print(result)  

             Ausland  Finanzen  Kultur  Lifestyle  Lokal  Politik  Sonstiges  \
Ausland          294       127      56        103     59      602        284   
Finanzen           0        72       1          1      0      104         27   
Kultur             0         0       0          0      0        0          0   
Lifestyle         86        41      43        107     25      444        167   
Lokal              0         8      19          2     78       33         16   
Politik            0        21      33          7     52       90         26   
Sonstiges         17         8      18         32      4       83         71   
Sport              0         0       0          0      0        0          0   
Technologie       81        52      37         68     29      211        167   
Wirtschaft         5         4       1          8      2       29         12   

             Sport  Technologie  Wirtschaft  
Ausland        160          105         296  
Finanzen         6         